<a href="https://colab.research.google.com/github/shwets1503/Sales_Prediction/blob/master/Know_your_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# import libraries
from datetime import datetime, timedelta
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division

import plotly.graph_objects as go

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
customer = pd.read_csv('../content/drive/My Drive/sales_data/customers_dataset.csv', engine='python')
orders_df = pd.read_csv('../content/drive/My Drive/sales_data2/olist_orders_dataset.csv', engine='python')
order_items = pd.read_csv('../content/drive/My Drive/sales_data2/olist_order_items_dataset.csv', engine='python')

In [47]:
customer.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [48]:
customer.shape

(99441, 5)

In [49]:
orders = pd.merge(customer, orders_df, how ='left', on='customer_id')

In [50]:
orders.shape

(99441, 12)

In [51]:
orders = pd.merge(orders, order_items,how ='left', on='order_id')

In [52]:
orders.shape

(113425, 18)

In [53]:
orders.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1.0,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,1.0,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,1.0,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,1.0,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,1.0,9391a573abe00141c56e38d84d7d5b3b,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25


In [54]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113425 entries, 0 to 113424
Data columns (total 18 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   customer_id                    113425 non-null  object 
 1   customer_unique_id             113425 non-null  object 
 2   customer_zip_code_prefix       113425 non-null  int64  
 3   customer_city                  113425 non-null  object 
 4   customer_state                 113425 non-null  object 
 5   order_id                       113425 non-null  object 
 6   order_status                   113425 non-null  object 
 7   order_purchase_timestamp       113425 non-null  object 
 8   order_approved_at              113264 non-null  object 
 9   order_delivered_carrier_date   111457 non-null  object 
 10  order_delivered_customer_date  110196 non-null  object 
 11  order_estimated_delivery_date  113425 non-null  object 
 12  order_item_id                 

In [33]:
orders[orders.customer_unique_id=='44bd7571a7dad968306f7776bd5a5745']

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
1772,7b04dd77487b6f8e2c13846684b2d742,44bd7571a7dad968306f7776bd5a5745,7500,santa isabel,SP,f4f43841ba1acc0c428e6d7c73658531,delivered,2018-02-19 21:22:19,2018-02-19 22:27:06,2018-02-20 22:19:19,2018-02-27 00:06:24,2018-03-15 00:00:00,1.0,92144cc00a16129494387f172aa54198,6b243f80ed07b10f0e8aa0f21a205f3c,2018-02-25 22:27:06,124.9,17.12
42439,e9987c23dd2a5898f664c89329981d77,44bd7571a7dad968306f7776bd5a5745,7500,santa isabel,SP,8745becbbbd36c57276dc93fed8bbe1c,delivered,2018-02-12 14:14:43,2018-02-12 15:00:26,2018-02-14 20:07:04,2018-02-16 10:22:35,2018-03-01 00:00:00,1.0,dd0dcee76f9c12fff4bc0eb641d57c7f,8b321bb669392f5163d04c59e235e066,2018-02-16 14:30:35,23.7,5.23
42440,e9987c23dd2a5898f664c89329981d77,44bd7571a7dad968306f7776bd5a5745,7500,santa isabel,SP,8745becbbbd36c57276dc93fed8bbe1c,delivered,2018-02-12 14:14:43,2018-02-12 15:00:26,2018-02-14 20:07:04,2018-02-16 10:22:35,2018-03-01 00:00:00,2.0,474bb6b54fc608ca71059a6c4f7ecda3,8b321bb669392f5163d04c59e235e066,2018-02-16 14:30:35,21.9,10.46
42441,e9987c23dd2a5898f664c89329981d77,44bd7571a7dad968306f7776bd5a5745,7500,santa isabel,SP,8745becbbbd36c57276dc93fed8bbe1c,delivered,2018-02-12 14:14:43,2018-02-12 15:00:26,2018-02-14 20:07:04,2018-02-16 10:22:35,2018-03-01 00:00:00,3.0,474bb6b54fc608ca71059a6c4f7ecda3,8b321bb669392f5163d04c59e235e066,2018-02-16 14:30:35,21.9,10.46


In [56]:
col = ['customer_unique_id', 'price', 'order_item_id', 'order_purchase_timestamp']
orders = orders[col]

Revenue = Active Customer Count * Order Count * Average Revenue per Order

In [57]:
#converting the type of Invoice Date Field from string to datetime.
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])

#creating YearMonth field for the ease of reporting and visualization
orders['InvoiceYearMonth'] = orders['order_purchase_timestamp'].map(lambda date: 100*date.year + date.month)

#calculate Revenue for each row and create a new dataframe with YearMonth - Revenue columns
orders['Revenue'] = orders['price'] * orders['order_item_id']
orders_revenue = orders.groupby(['InvoiceYearMonth'])['Revenue'].sum().reset_index()
orders_revenue.head()

,InvoiceYearMonth,Revenue
0,201609,435.23
1,201610,56103.79
2,201612,10.90
3,201701,142077.30
4,201702,269786.66


In [58]:
#create a dataframe contaning CustomerID and first purchase date
orders_min_purchase = orders.groupby('customer_unique_id').order_purchase_timestamp.min().reset_index()
orders_min_purchase.columns = ['customer_unique_id','MinPurchaseDate']
orders_min_purchase['MinPurchaseYearMonth'] = orders_min_purchase['MinPurchaseDate'].map(lambda date: 100*date.year + date.month)

#merge first purchase date column to our main dataframe (tx_uk)
orders = pd.merge(orders, orders_min_purchase, on='customer_unique_id')

In [59]:
#create a column called User Type and assign Existing 
#if User's First Purchase Year Month before the selected Invoice Year Month
orders['UserType'] = 'New'
orders.loc[orders['InvoiceYearMonth']>orders['MinPurchaseYearMonth'],'UserType'] = 'Existing'

#calculate the Revenue per month for each user type
orders_user_type_revenue = orders.groupby(['InvoiceYearMonth','UserType'])['Revenue'].sum().reset_index()
orders.head()

,customer_unique_id,price,order_item_id,order_purchase_timestamp,InvoiceYearMonth,Revenue,MinPurchaseDate,MinPurchaseYearMonth,UserType
0,861eff4711a542e4b93843c6dd7febb0,124.99,1.0,2017-05-16 15:05:35,201705,124.99,2017-05-16 15:05:35,201705,New
1,290c77bc529b7ac935b93aa66c333dc3,289.00,1.0,2018-01-12 20:48:24,201801,289.00,2018-01-12 20:48:24,201801,New
2,060e732b5b29e8181a18229c7b0b2b5e,139.94,1.0,2018-05-19 16:07:45,201805,139.94,2018-05-19 16:07:45,201805,New
3,259dac757896d24d7702b9acbbff3f3c,149.94,1.0,2018-03-13 16:06:38,201803,149.94,2018-03-13 16:06:38,201803,New
4,345ecd01c38d18a9036ed96c73b8d066,230.00,1.0,2018-07-29 09:51:30,201807,230.00,2018-07-29 09:51:30,201807,New


In [63]:
orders[orders['order_purchase_timestamp']>orders['MinPurchaseDate']]

,customer_unique_id,price,order_item_id,order_purchase_timestamp,InvoiceYearMonth,Revenue,MinPurchaseDate,MinPurchaseYearMonth,UserType
6,4c93744516667ad3b8f1fb645a3116a4,246.62,1.0,2017-10-15 12:44:13,201710,246.62,2017-09-14 18:14:31,201709,Existing
10,1175e95fb47ddff9de6b2b06188f7e0d,29.99,1.0,2018-01-18 12:35:45,201801,29.99,2018-01-18 12:35:44,201801,New
16,295c05e81917928d76245e842748184d,220.98,1.0,2018-06-06 11:59:53,201806,220.98,2018-03-07 15:57:14,201803,Existing
37,e9dd12dca17352644a959d9dea133935,44.99,1.0,2018-04-18 09:42:53,201804,44.99,2018-03-05 18:12:15,201803,Existing
41,e079b18794454de9d2be5c12b4392294,56.99,1.0,2017-06-14 18:52:29,201706,56.99,2017-06-14 18:31:54,201706,New
...,...,...,...,...,...,...,...,...,...
109341,76f7d40948174f5cde0fcf8e6147b59d,74.90,1.0,2018-04-29 22:59:41,201804,74.90,2017-12-30 00:06:41,201712,Existing
109342,76f7d40948174f5cde0fcf8e6147b59d,74.90,2.0,2018-04-29 22:59:41,201804,149.80,2017-12-30 00:06:41,201712,Existing
109859,88fcb680dc8cd2e540010c0750b78242,56.50,1.0,2018-01-19 08:02:06,201801,56.50,2017-04-24 10:53:12,201704,Existing
110342,2dcbfb99d95c7ba6eb992a98627f45b8,85.99,1.0,2017-12-03 20:37:09,201712,85.99,2017-11-28 01:16:27,201711,Existing


In [61]:
orders.UserType.value_counts()

New         110963
Existing      2462
Name: UserType, dtype: int64

In [167]:
#X and Y axis inputs for Plotly graph. We use Scatter for line graphs
plot_data = [
    go.Scatter(
        x=orders_revenue['InvoiceYearMonth'],
        y=orders_revenue['Revenue'],
        mode = 'lines+markers'
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Montly Revenue', width=900, height=500
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

In [168]:
#using pct_change() function to see monthly percentage change
orders_revenue['MonthlyGrowth'] = orders_revenue['Revenue'].pct_change()

#showing first 5 rows
orders_revenue.head()

,InvoiceYearMonth,Revenue,MonthlyGrowth
0,201609,435.23,NaN
1,201610,56103.79,127.906073
2,201612,10.90,-0.999806
3,201701,142077.30,13033.614679
4,201702,269786.66,0.898872


In [169]:
#visualization - line graph
plot_data = [
    go.Scatter(
        x=orders_revenue.query("InvoiceYearMonth > 201701")['InvoiceYearMonth'],
        y=orders_revenue.query("InvoiceYearMonth > 201701")['MonthlyGrowth'],
        mode = 'lines+markers'
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Montly Growth Rate', width=900, height=500
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

In [170]:
#creating a new dataframe with UK customers only
#tx_uk = tx_data.query("Country=='United Kingdom'").reset_index(drop=True)

#creating monthly active customers dataframe by counting unique Customer IDs
orders_monthly_active = orders.groupby('InvoiceYearMonth')['customer_id'].nunique().reset_index()

#print the dataframe
orders_monthly_active.head()

,InvoiceYearMonth,customer_id
0,201609,3
1,201610,308
2,201612,1
3,201701,789
4,201702,1733


In [171]:
#plotting the output
plot_data = [
    go.Bar(
        x=orders_monthly_active['InvoiceYearMonth'],
        y=orders_monthly_active['customer_id'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Active Customers',width=900, height=500
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

In [172]:
#create a new dataframe for no. of order by using quantity field
orders_monthly_sales = orders.groupby('InvoiceYearMonth')['order_item_id'].sum().reset_index()

#print the dataframe
orders_monthly_sales.head()

,InvoiceYearMonth,order_item_id
0,201609,10
1,201610,457
2,201612,1
3,201701,1318
4,201702,2252


In [173]:
#plot
plot_data = [
    go.Bar(
        x=orders_monthly_sales['InvoiceYearMonth'],
        y=orders_monthly_sales['order_item_id'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Total # of Order',width=900, height=500
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

In [174]:
# create a new dataframe for average revenue by taking the mean of it
orders_monthly_order_avg = orders.groupby('InvoiceYearMonth')['Revenue'].mean().reset_index()

#print the dataframe
orders_monthly_order_avg.head()

,InvoiceYearMonth,Revenue
0,201609,72.538333
1,201610,154.555895
2,201612,10.900000
3,201701,148.772042
4,201702,138.281220


In [175]:
#plot the bar chart
plot_data = [
    go.Bar(
        x=orders_monthly_order_avg['InvoiceYearMonth'],
        y=orders_monthly_order_avg['Revenue'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Order Average',width=900, height=500
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

In [176]:
#create a dataframe contaning CustomerID and first purchase date
orders_min_purchase = orders.groupby('customer_id').order_purchase_timestamp.min().reset_index()
orders_min_purchase.columns = ['customer_id','MinPurchaseDate']
orders_min_purchase['MinPurchaseYearMonth'] = orders_min_purchase['MinPurchaseDate'].map(lambda date: 100*date.year + date.month)

#merge first purchase date column to our main dataframe (tx_uk)
orders = pd.merge(orders, orders_min_purchase, on='customer_id')

In [155]:
orders.groupby('customer_id').order_purchase_timestamp.min().reset_index()

,customer_id,order_purchase_timestamp
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26
1,000161a058600d5901f007fab4c27140,2017-07-16 09:40:32
2,0001fd6190edaaf884bcaf3d49edf079,2017-02-28 11:06:43
3,0002414f95344307404f0ace7a26f1d5,2017-08-16 13:09:20
4,000379cdec625522490c315e70c7a9fb,2018-04-02 13:42:17
...,...,...
98661,fffcb937e9dd47a13f05ecb8290f4d3e,2018-03-17 00:55:27
98662,fffecc9f79fd8c764f843e9951b11341,2018-03-29 16:59:26
98663,fffeda5b6d849fbd39689bb92087f431,2018-05-22 13:36:02
98664,ffff42319e9b2d713724ae527742af25,2018-06-13 16:57:05


In [182]:
orders

,customer_id,price,order_item_id,order_purchase_timestamp,InvoiceYearMonth,Revenue,MinPurchaseDate,MinPurchaseYearMonth,UserType
0,9ef432eb6251297304e76186b10a928d,29.99,1,2017-10-02 10:56:33,201710,29.99,2017-10-02 10:56:33,201710,New
1,b0830fb4747a6c6d20dea0b8c802d7ef,118.70,1,2018-07-24 20:41:37,201807,118.70,2018-07-24 20:41:37,201807,New
2,41ce2a54c0b03bf3443c3d931a367089,159.90,1,2018-08-08 08:38:49,201808,159.90,2018-08-08 08:38:49,201808,New
3,f88197465ea7920adcdbec7375364d82,45.00,1,2017-11-18 19:28:06,201711,45.00,2017-11-18 19:28:06,201711,New
4,8ab97904e6daea8866dbdbc4fb7aad2c,19.90,1,2018-02-13 21:18:39,201802,19.90,2018-02-13 21:18:39,201802,New
...,...,...,...,...,...,...,...,...,...
112645,1fca14ff2861355f6e5f14306ff977a7,174.90,1,2018-02-06 12:58:58,201802,174.90,2018-02-06 12:58:58,201802,New
112646,1aa71eb042121263aafbe80c1b562c9c,205.99,1,2017-08-27 14:46:43,201708,205.99,2017-08-27 14:46:43,201708,New
112647,b331b74b18dc79bcdf6532d51e1637c1,179.99,1,2018-01-08 21:28:27,201801,179.99,2018-01-08 21:28:27,201801,New
112648,b331b74b18dc79bcdf6532d51e1637c1,179.99,2,2018-01-08 21:28:27,201801,359.98,2018-01-08 21:28:27,201801,New


In [178]:
#create a column called User Type and assign Existing 
#if User's First Purchase Year Month before the selected Invoice Year Month
orders['UserType'] = 'New'
orders.loc[orders['InvoiceYearMonth']>orders['MinPurchaseYearMonth'],'UserType'] = 'Existing'

#calculate the Revenue per month for each user type
orders_user_type_revenue = orders.groupby(['InvoiceYearMonth','UserType'])['Revenue'].sum().reset_index()
orders.head()

,customer_id,price,order_item_id,order_purchase_timestamp,InvoiceYearMonth,Revenue,MinPurchaseDate,MinPurchaseYearMonth,UserType
0,9ef432eb6251297304e76186b10a928d,29.99,1,2017-10-02 10:56:33,201710,29.99,2017-10-02 10:56:33,201710,New
1,b0830fb4747a6c6d20dea0b8c802d7ef,118.70,1,2018-07-24 20:41:37,201807,118.70,2018-07-24 20:41:37,201807,New
2,41ce2a54c0b03bf3443c3d931a367089,159.90,1,2018-08-08 08:38:49,201808,159.90,2018-08-08 08:38:49,201808,New
3,f88197465ea7920adcdbec7375364d82,45.00,1,2017-11-18 19:28:06,201711,45.00,2017-11-18 19:28:06,201711,New
4,8ab97904e6daea8866dbdbc4fb7aad2c,19.90,1,2018-02-13 21:18:39,201802,19.90,2018-02-13 21:18:39,201802,New


In [179]:
orders.UserType.value_counts()

New    112650
Name: UserType, dtype: int64

In [ ]:
#filtering the dates and plot the result
tx_user_type_revenue = tx_user_type_revenue.query("InvoiceYearMonth != 201012 and InvoiceYearMonth != 201112")
plot_data = [
    go.Scatter(
        x=tx_user_type_revenue.query("UserType == 'Existing'")['InvoiceYearMonth'],
        y=tx_user_type_revenue.query("UserType == 'Existing'")['Revenue'],
        name = 'Existing'
    ),
    go.Scatter(
        x=tx_user_type_revenue.query("UserType == 'New'")['InvoiceYearMonth'],
        y=tx_user_type_revenue.query("UserType == 'New'")['Revenue'],
        name = 'New'
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='New vs Existing'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

NameError: ignored